In [20]:
import pandas as pd
import ee
import geetools
import folium
import time
from folium import plugins
import Leaf_tools
from geopandas import GeoDataFrame
from shapely.geometry import shape
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AY0e-g6D0vCMI-t13FxseE13k292T8J8ZFb1k1AWUsnMqAfiC1ZYk2N2HlE



Successfully saved authorization token.


**import data**

In [22]:
tiles_CCRS = ee.FeatureCollection("users/ganghong/TilesSampling_Sub")
print (tiles.size().getInfo())
tiles_Bel = ee.FeatureCollection('users/ganghong/TilesSampling_Belmanip_Sub')
print (tiles_Bel.size().getInfo())
ezone = ee.FeatureCollection("users/ganghong/NA_Forests")
fapar_CCRS = ee.ImageCollection("users/ganghong/FAPAR_CCRS")
fcover_CCRS = ee.ImageCollection("users/ganghong/FCOVER")
LAI_CCRS = ee.ImageCollection("users/ganghong/LAI_CCRS")
fapar_Bel = ee.ImageCollection("users/ganghong/FAPAR_Bel")
fcover_Bel = ee.ImageCollection("users/ganghong/FCOVER_Bel")
LAI_Bel = ee.ImageCollection("users/ganghong/LAI_Bel")
print (fapar_CCRS.size().getInfo())
print (fcover_CCRS.size().getInfo())
print (LAI_CCRS.size().getInfo())
print (fapar_Bel.size().getInfo())
print (fcover_Bel.size().getInfo())
print (LAI_Bel.size().getInfo())

28
35
1764
1823
1680
2175
2202
2092


**functions definition**

In [23]:
basemaps = {
'Google Maps': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Maps',
overlay = True,
control = True
),
'Google Satellite': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Satellite',
overlay = True,
control = True
)}

## display layers
def add_ee_layer(self, ee_object, vis_params, name):
 try:
  if isinstance(ee_object, ee.image.Image):
   map_id_dict = ee.Image(ee_object).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.imagecollection.ImageCollection):
   ee_object_new = ee_object.mosaic()
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name, 
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.geometry.Geometry):
   folium.GeoJson(
   data = ee_object.getInfo(),
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object,ee.featurecollection.FeatureCollection):
   ee_object_new = ee.Image().paint(ee_object, 0, 2)
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

 except:
  print("Could not display {}".format(name))

#date conversion
def DatetoSystemtime(datelist):
    geestamp=[]
    for date in datelist:       
        date = pd.to_datetime(date, format='%Y-%m-%d')       
       # print (date)
        year_day = pd.Timestamp(year=date.year, month=1, day=1)       
        doy = (date - year_day).days + 1     ## add three digits ###   
        hour = '12'
        year=date.year
        timestamp = str(year) + str(doy) + hour
        # convert time stamp to gee format
        gee_timestamp = int(pd.to_datetime(timestamp,format='%Y%j%H').value // 10**6)
        #print (gee_timestamp)
        geestamp.append(gee_timestamp)        
    return geestamp    

#remove properties 
def removeProperty(feat): 
    properties = feat.propertyNames()
    selectProperties = properties.filter(ee.Filter.eq('item', 'MGRS_TILE')) 
    return feat.select(selectProperties)


def polyInterFeatureCollection(tile):    
    tile = ee.Feature(tile)
    intersects =ezone.map(polygonintersect)   
    #intersects =ezone.map(lambda feat1:polygonintersect(*feat1))
    sortedintersect=intersects.sort('Intersect',false)
    return sortedinterset.first()

#featurecollection intersection
def featurecollection_interset(tile, resultlist):
    resultlist = ee.List(resultlist)
    tile = ee.Feature(tile)
    def polygonintersect(zone):    
        zone = ee.Feature(zone)   
        intersection = tile.intersection(zone, ee.ErrorMargin(1));
        return intersection.set({'ezone': zone.get('EZMAP_ZONE')}).set({'Intersect': intersection.area().divide(1000 * 1000)})
    intersection = zoneList.map(polygonintersect)
    sortedintersect=ee.FeatureCollection(intersection).sort('Intersect', False)
    return resultlist.add(sortedintersect.first())

#land cover frequency
def calculateFreq(tile,list):
    list=ee.List(list)
    frequency = NALCMS_forest.mosaic().reduceRegion(
       reducer=ee.Reducer.frequencyHistogram(),
       maxPixels=2e9,
       scale=500,
       geometry=tile.geometry()
      )
    tempobj= frequency.get("landcover")
    okeys = ee.Dictionary(tempobj).keys()
    ovals = ee.Dictionary(tempobj).values()
    minmax = ee.Dictionary(ovals.reduce(ee.Reducer.minMax()))
    maxval = ee.Number(minmax.get('max'))
    maxind = ovals.indexOf(maxval)
    maxkey = okeys.get(maxind)
    tile= tile.set({'landcover': maxkey})
    return list.add(tile)

#feature to dataframe
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']
    dictarr = []
    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        #attr['geometry'] = f['geometry']  # GeoJSON Feature!
        # attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)
        df = GeoDataFrame(dictarr)
        # Convert GeoJSON features to shape
        #df['geometry'] = map(lambda s: shape(s), df.geometry)
    return df

#add date to a feature
def datelist(img):
    imgdate=ee.Date(img.get("system:time_start"))
    return ee.Feature(None, {'date': imgdate.format('YYYY-MM-dd')})
 
#bit wsie operation  
def bitwiseExtract(value, fromBit, toBit):
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)

#modis product bit mask
def bitmask(image):
    qa = image.select('FparLai_QC')
    good = bitwiseExtract(qa, 0,0) # returns 0 for good quality
    return image.updateMask(ee.Image(good).eq(0))  ## needs to be inverted to mask pixels with bad quality, 0 means good quality , pass the mask   

## loops for stats based LAI
def applyS2(tile):
    tile=ee.FeatureCollection(tile)
    tilename=ee.Feature(tile).get('MGRS_TILE')
    def statscal(img):    
        sts=img.select('estimateLAI','errorLAI').reduceRegion(
        reducer= reducers,
        geometry= tile.geometry(),
        scale= MODISscale,
        maxPixels= 1e9
        )        
        return ee.Feature(None, {'tile': tilename, 'date':img.get('date'), 'LAI_mean':sts.get('estimateLAI_mean'),'LAI_std':sts.get('estimateLAI_stdDev'),'number of samples ':sts.get('estimateLAI_count'),'LAIunc_mean':sts.get('errorLAI_mean')}) 
    feat=dataS2matchedMODIS.filterBounds(tile.geometry()).map(statscal)
    return feat    
  

**dominant ezone**

In [24]:
## remove properties not required in each feature, only MGRS_TILE kept
newTiles = tiles_CCRS.map(removeProperty)
#print (ee.Feature(newTiles.toList(newTiles.size()).get(0)).propertyNames().getInfo())
zoneList = ezone.toList(ezone.size())
tileIntersect = newTiles.iterate(featurecollection_interset,ee.List([]))
tileIntersect = ee.FeatureCollection(ee.List(tileIntersect).flatten()) 
#print (ee.Feature(tileIntersect.toList(tileIntersect.size()).get(0)).propertyNames().getInfo())
#print (tileIntersect.size().getInfo())
folium.Map.add_ee_layer = add_ee_layer
my_map = folium.Map(location=[56,-106],
                    zoom_start=4)
my_map.add_ee_layer(tileIntersect, {}, 'intersectarea')
my_map.add_child(folium.LayerControl())
display(my_map)

**dominant land cover**

In [25]:
##Define Forest Cover map to use - should be a global constant
NALCMS_forest = ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles')\
.map(lambda image:image.select("b1").rename("landcover"))\
.merge(ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global")\
.map(lambda image:image.select("discrete_classification").remap([0,20,30,40,50,60,70,80,90,100,111,112,113,114,115,116,121,122,123,124,125,126,200],[0,8,10,15,17,16,19,18,14,13,1,3,1,5,6,6,2,4,2,5,6,6,18],0).toUint8().rename("landcover")))

tilestats= ee.FeatureCollection(ee.List(tileIntersect.iterate(calculateFreq,ee.List([]))).flatten())
#print (ee.Feature(secondtile.toList(secondtile.size()).get(0)).propertyNames().getInfo())
#print (ee.Feature(secondtile.toList(secondtile.size()).get(17)).get('landcover').getInfo())
tilesframe=fc2df(tilestats)
print(tilesframe)
tilesframe.to_csv('tiles_summary.csv')

       Intersect MGRS_TILE                         ezone landcover
0   11067.035589     09WXP        Boreal tundra woodland         1
1   11999.771104     10VEL        Boreal tundra woodland         1
2   11995.691198     13VDH        Boreal tundra woodland         8
3    9276.457129     18WWT                         Polar        13
4   12011.348328     11UNA      Boreal coniferous forest         1
5   11992.307326     14UPE      Boreal coniferous forest         1
6   11983.720807     16UEE        Boreal tundra woodland         8
7   11388.776629     21UWQ      Boreal coniferous forest         6
8    7620.895955     21TYN      Boreal coniferous forest         1
9    8513.159088     10TFP     Temperate mountain system         1
10  12027.525684     11TPM     Temperate mountain system         1
11  11784.234164     15TUN  Temperate continental forest         5
12  12017.904878     15TYL  Temperate continental forest         5
13   6347.433752     18TVP     Temperate mountain system      

**ImageCollection**

In [28]:
year_start=2019
year_end=2019
month_start=4
month_end=9
mapBounds= ee.FeatureCollection(ee.Feature(newTiles.toList(newTiles.size()).get(0))).geometry()
dataMODIS =ee.ImageCollection('MODIS/006/MCD15A3H').filterBounds(mapBounds).filter(ee.Filter.calendarRange(year_start, year_end, 'year')).filter(ee.Filter.calendarRange(month_start, month_end, 'month'))
dataS2=ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(mapBounds).filter(ee.Filter.calendarRange(year_start, year_end, 'year')).filter(\
        ee.Filter.calendarRange(month_start, month_end, 'month')).filterMetadata("CLOUDY_PIXEL_PERCENTAGE", 'less_than',60) 
dateMODIS=dataMODIS.map(lambda img: ee.Feature(None, {'date': ee.Date(img.get("system:time_start")).format('YYYY-MM-dd')})).distinct('date').aggregate_array('date')
dataS2=Leaf_tools.Leaf_tools(mapBounds, 2019)
dateS2=dataS2.map(lambda img: ee.Feature(None, {'date': ee.Date(img.get("system:time_start")).format('YYYY-MM-dd')})).distinct('date').aggregate_array('date')
dateCLGS=LAI_CCRS.filterBounds(mapBounds).map(lambda img: ee.Feature(None, {'date': ee.Date(img.get("system:time_start")).format('YYYY-MM-dd')})).distinct('date').aggregate_array('date')
print (dateMODIS.size().getInfo())  
print (dateS2.size().getInfo())  
print (dateCLGS.size().getInfo()) 
#print (dateS2.getInfo())
#print (dateMODIS.getInfo())

datelistMODIS_S2=list(set(dateS2.getInfo()).intersection(dateMODIS.getInfo())) ## matched between MODIS and S2
print("matched dates between MODIS and S2")
print (datelistMODIS_S2)
datelistMODIS_CLGS=list(set(dateCLGS.getInfo()).intersection(dateMODIS.getInfo())) ## matched between MODIS and CLGS
print("matched dates between MODIS and CLGS")
print (datelistMODIS_CLGS)

46
124
60
matched dates between MODIS and S2
['2019-04-19', '2019-05-01', '2019-08-01', '2019-04-11', '2019-08-21', '2019-06-26', '2019-07-08', '2019-07-20', '2019-06-06', '2019-09-06', '2019-07-28', '2019-05-09', '2019-06-30', '2019-04-03', '2019-06-02', '2019-08-05', '2019-09-18', '2019-08-29', '2019-06-18', '2019-05-13', '2019-08-09', '2019-04-23', '2019-05-17', '2019-06-10', '2019-07-12', '2019-09-26', '2019-04-27', '2019-05-21', '2019-06-22', '2019-09-10', '2019-07-16', '2019-08-25']
matched dates between MODIS and CLGS
['2019-09-30', '2019-07-20', '2019-06-30', '2019-06-10']


**date related processing**

In [29]:
## add date as a property
dataMODIS=dataMODIS.map(lambda img:img.set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd")))
dataCLGS=LAI_Bel.map(lambda img:img.set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd")))
dataS2=dataS2.map(lambda img:img.set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd")))
### match bettween MODIS and S2
dataS2matchedMODIS=dataS2.filter(ee.Filter.inList('date', ee.List(datelistMODIS_S2))).map(lambda img:img.updateMask(img.eq(0)))
dataMODISmatchedS2=dataMODIS.filter(ee.Filter.inList('date', ee.List(datelistMODIS_S2))).map(bitmask)
print (dataS2matchedMODIS.size().getInfo())  
print (dataMODISmatchedS2.size().getInfo()) 
### match bettween MODIS and CLGS
dataCLGSmatchedMODIS=dataCLGS.filter(ee.Filter.inList('date', ee.List(datelistMODIS_CLGS))).map(lambda img:img.updateMask(img.eq(1)))
dataMODISmatchedS2=dataMODIS.filter(ee.Filter.inList('date', ee.List(datelistMODIS_CLGS))).map(bitmask)
print (dataCLGSmatchedMODIS.size().getInfo()) 
print (dataMODISmatchedS2.size().getInfo()) 

181
32
140
4


**Statistic analysis: mean, std, count**

In [30]:
reducers = ee.Reducer.mean().combine(
  reducer2= ee.Reducer.stdDev(),
  sharedInputs= True
).combine(
  reducer2= ee.Reducer.count(),
  sharedInputs= True
)

testtile=ee.FeatureCollection(ee.Feature(tileIntersect.toList(tiles.size()).get(0)))
MODISscale=dataMODISmatchedS2.first().projection().nominalScale()
S2output=testtile.map(applyS2).flatten()
#S2output=tileIntersect.map(applyS2).flatten()
#S2list=fc2df(S2output)
#print(S2list)
print(S2output.size().getInfo())
#print (ee.Feature(S2output.toList(S2output.size()).get(0)).getInfo())
#print (ee.Feature(S2output.toList(S2output.size()).get(0)).propertyNames().getInfo())
#print (ee.Feature(secondtile.toList(secondtile.size()).get(17)).get('landcover').getInfo())


181
